In [2]:
import pandas , requests, json, time
import multiprocessing, os
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy

In [4]:
import re

In [5]:
import glob

In [74]:

def routing_network(lat_o, lon_o, lat_d, lon_d):
    headers = { "content-type": "application/json" }
    start=[lon_o, lat_o]
    end=[lon_d, lat_d]
    priorityJson = {'road_class': {'motorway': 0.00, '*': 1}}

    
    
    route_url = 'http://localhost:8989/route-custom?'
    route = requests.post(url=route_url,headers=headers, json={'profile':'bikef', 'points':[start,end], 'points_encoded':False, 'elevation':False, 'priority': priorityJson,
                                                               'max_speed_fallback': 0.01}).json()
    try:
        print(route)
        coordinates = route['paths'][0]['points']['coordinates']
        distance = route['paths'][0]['distance']
        duration = route['paths'][0]['time']
        return (coordinates, distance, duration)
    except:

        return (route)
    #     print(f"http://127.0.0.1:8989/route?point={lat_o}%2C{lon_o}&point={lat_d}%2C{lon_d}&locale=en-US&elevation=false&points_encoded=false&profile=bike&use_miles=false&layer=Omniscale")

        print ([],[],[])

In [75]:
b=routing_network(42.37777,-70.987247,42.379241,-70.987843)

{'hints': {'visited_nodes.sum': 34, 'visited_nodes.average': 34.0}, 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors'], 'took': 9}, 'paths': [{'distance': 214.459, 'weight': 77220.181343, 'time': 77205169, 'transfers': 0, 'points_encoded': False, 'bbox': [-70.987937, 42.377774, -70.987247, 42.379234], 'points': {'type': 'LineString', 'coordinates': [[-70.987247, 42.377774], [-70.987306, 42.377774], [-70.987365, 42.377804], [-70.987655, 42.377854], [-70.98784, 42.37806], [-70.987802, 42.378081], [-70.9878, 42.378155], [-70.987845, 42.378188], [-70.987891, 42.378191], [-70.987848, 42.378622], [-70.987875, 42.37869], [-70.98792, 42.378733], [-70.987875, 42.378741], [-70.987837, 42.378767], [-70.98791, 42.378828], [-70.987886, 42.37892], [-70.987937, 42.379185], [-70.987899, 42.379229], [-70.987842, 42.379234]]}, 'instructions': [{'annotation_text': 'get off the bike', 'distance': 35.312, 'heading': 270.64, 'sign': 0, 'annotation_importance': 1, 'interval': [0, 3], 'text'

In [76]:
b

([[-70.987247, 42.377774],
  [-70.987306, 42.377774],
  [-70.987365, 42.377804],
  [-70.987655, 42.377854],
  [-70.98784, 42.37806],
  [-70.987802, 42.378081],
  [-70.9878, 42.378155],
  [-70.987845, 42.378188],
  [-70.987891, 42.378191],
  [-70.987848, 42.378622],
  [-70.987875, 42.37869],
  [-70.98792, 42.378733],
  [-70.987875, 42.378741],
  [-70.987837, 42.378767],
  [-70.98791, 42.378828],
  [-70.987886, 42.37892],
  [-70.987937, 42.379185],
  [-70.987899, 42.379229],
  [-70.987842, 42.379234]],
 214.459,
 77205169)

In [82]:
def extract_point(matched_point):
    
    lon, lat = re.match(r"(.*)\((.*)\)" ,matched_point).groups()[-1].split(" ")
    return lon, lat

In [48]:
def routing(lat_o, lon_o, lat_d, lon_d):   ## Requests for getting route using foot
    resj = json.loads(requests.get(
        f"http://127.0.0.1:8989/route_custom?point={lat_o}%2C{lon_o}&point={lat_d}%2C{lon_d}&locale=en-US&elevation=false&max_speed_fallback=0.01&points_encoded=false&profile=bikef&use_miles=false&layer=Omniscale"
        ).content)

    try:
        print(resj)
        coordinates = resj['paths'][0]['points']['coordinates']
        distance = resj['paths'][0]['distance']
        duration = resj['paths'][0]['time']
        return (coordinates, distance, duration)
    except:
        print(resj)

        return ([],[],[])

In [49]:
routing(42.378369,-70.987515,42.379229,-70.987751)

{'code': 404, 'message': 'HTTP 404 Not Found'}
{'code': 404, 'message': 'HTTP 404 Not Found'}


([], [], [])

In [48]:
def route_modeler(df): 
    ## modeled route by foot profile
    
    df[['f_coordinates', 'f_distance', 'f_duration']] = df.apply(lambda row: pandas.Series(routing_network(row['lat_o'], row['lon_o'], row['lat_d'], row['lon_d'])), axis=1)
    df['dist_ratio'] = df['dist_calc']/df['f_distance']
    print(df)
    
    return df

In [89]:
def processor(df, name):
    
    routed_df = route_modeler(df)
    df.to_csv(f"C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_321_384\\json_files\\{name}.csv", index = False)
    route_linesCollection = csv_tojson(routed_df)
    json_temp = json.dumps(route_linesCollection)
    f = open(f"C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_321_384\\json_files\\{name}.json","w")
    f.write(json_temp)
    f.close()
    print('done')


def csv_tojson(df):
    linesCollection = dict(type="FeatureCollection", features=[])
    for i in range(len(df)):
        print(i)
        propertDict = dict(
            trip_id= df['trip_id'].tolist()[i],
            dist_calc = df['dist_calc'].tolist()[i],
            f_distance = df['f_distance'].tolist()[i],
            dist_ratio = df['dist_ratio'].tolist()[i],
            propulsion_type = df['propulsion_type'].tolist()[i],
            lat_o = df['lat_o'].tolist()[i],
            lon_o = df['lon_o'].tolist()[i],
            lat_d = df['lat_d'].tolist()[i],
            lon_d = df['lon_d'].tolist()[i]
 )
        linegeo = dict(type="LineString",coordinates=[])
        for f in df['f_coordinates'].tolist()[i]:
            # print(f)
            linegeo['coordinates'].append(f)
        linefeature = dict(type="feature", geometry=linegeo, properties=propertDict)
        linesCollection['features'].append(linefeature)
    return linesCollection

In [63]:
bikes_n = pandas.read_csv("C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_321_384\\err70_matchedStartEnd.csv")

In [83]:
bikes_n[['lon_o', 'lat_o']] = bikes_n.apply(lambda row: pandas.Series(extract_point(row['matched_start'])), axis=1)
bikes_n[['lon_d', 'lat_d']] = bikes_n.apply(lambda row: pandas.Series(extract_point(row['matched_end'])), axis=1)

In [85]:
bikes_trps_f_n = bikes_n[bikes_n['trip_id'].isin(faulty_trips)]

In [86]:
bikes_trps_f_n

,trip_id,dist_calc,od_line_dist,matching_error,propulsion_type,matched_start,matched_end,lon_o,lat_o,lon_d,lat_d
163,61a0aed6-fc45-4f5d-b767-317765adb109,1019,879,6.98,"['human', 'electric_assist']",POINT(-71.206568 42.3339009999999),POINT(-71.205802 42.3417879999999),-71.206568,42.3339009999999,-71.205802,42.3417879999999
482,2d1dbdc0-ee19-4a3d-b045-4bfbf5abec34,812,533,11.46,"['human', 'electric_assist']",POINT(-71.190986 42.3728499999999),POINT(-71.191851 42.3772389999999),-71.190986,42.3728499999999,-71.191851,42.3772389999999
531,12fd115c-7ba2-4ee9-b024-9c9d1b3f0bab,478,477,4.46,"['human', 'electric_assist']",POINT(-71.044208 42.4306549999999),POINT(-71.049989 42.4302609999999),-71.044208,42.4306549999999,-71.049989,42.4302609999999
571,7dc12fcd-a9ce-4868-b77e-9a04104f0103,421,355,0.49,"['human', 'electric_assist']",POINT(-71.004841 42.4063799999999),POINT(-71.002436 42.4090109999999),-71.004841,42.4063799999999,-71.002436,42.4090109999999
572,3f76c7a7-5f3a-4228-b717-d68f7ab790cf,549,414,9.05,"['human', 'electric_assist']",POINT(-71.199194 42.3505069999999),POINT(-71.202873 42.3480399999999),-71.199194,42.3505069999999,-71.202873,42.3480399999999
...,...,...,...,...,...,...,...,...,...,...,...
9687,8157ef02-1e9c-4ee6-b04c-f184bea860c2,643,509,3.94,"['human', 'electric_assist']",POINT(-71.044036 42.4603649999999),POINT(-71.048302 42.4636679999999),-71.044036,42.4603649999999,-71.048302,42.4636679999999
9713,0fb76036-afc3-480e-b027-73408c7487e6,429,365,3.14,"['human', 'electric_assist']",POINT(-71.182327 42.3661819999999),POINT(-71.18527 42.3686199999999),-71.182327,42.3661819999999,-71.18527,42.3686199999999
9845,2ae6db8f-3dea-4f9e-b494-d78ac9519def,1084,905,6.41,"['human', 'electric_assist']",POINT(-71.021418 42.2697129999999),POINT(-71.028883 42.2753309999999),-71.021418,42.2697129999999,-71.028883,42.2753309999999
9892,769c35a5-0f43-45ca-bc04-26c9223fe5fd,580,401,17.84,"['human', 'electric_assist']",POINT(-71.183384 42.3635119999999),POINT(-71.17924 42.3616489999998),-71.183384,42.3635119999999,-71.17924,42.3616489999998


In [90]:
processor(bikes_trps_f_n.head(2), 'bikes_trps_f_3_2')

                                  trip_id  dist_calc  od_line_dist  \
163  61a0aed6-fc45-4f5d-b767-317765adb109       1019           879   
482  2d1dbdc0-ee19-4a3d-b045-4bfbf5abec34        812           533   

     matching_error               propulsion_type  \
163            6.98  ['human', 'electric_assist']   
482           11.46  ['human', 'electric_assist']   

                          matched_start                         matched_end  \
163  POINT(-71.206568 42.3339009999999)  POINT(-71.205802 42.3417879999999)   
482  POINT(-71.190986 42.3728499999999)  POINT(-71.191851 42.3772389999999)   

          lon_o             lat_o       lon_d             lat_d  \
163  -71.206568  42.3339009999999  -71.205802  42.3417879999999   
482  -71.190986  42.3728499999999  -71.191851  42.3772389999999   

                                         f_coordinates  f_distance  \
163  [[-71.206565, 42.3339], [-71.20651, 42.334034]...    1019.575   
482  [[-71.190986, 42.37285], [-71.190935, 42.372

In [ ]:
datatypes = dict()
for c in ddf.columns:
    print(c)
    datatypes.update({
        f"{c}": str
    })

In [18]:
ddf = pandas.read_csv("C:\\Users\\bitas\\folders\\Research\\lime\peter\data\\detour\\cr_00_140\\csv_files\\50.csv", dtype=datatypes)

In [26]:
for c in ddf.columns:
    print(c)

trip_id
dist_calc
od_line_dist
matching_error
propulsion_type
matched_start
matched_end
lon_o
lat_o
lon_d
lat_d
f_coordinates
f_distance
f_duration
dist_ratio
safe_ratio


In [49]:
DATA_PATH = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_00_140\\csv_files\\*.csv'
DATA_PATH2 = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_141_320\\csv_files\\*.csv'
DATA_PATH3 = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_321_384\\csv_files\\*.csv'
DATA_PATH4 = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_384_500\\csv_files\\*.csv'


In [50]:
files_list = []


In [51]:
for fname in glob.glob(DATA_PATH):
    files_list.append(pandas.read_csv(fname, dtype=datatypes))
#     print(fname)
for fname2 in glob.glob(DATA_PATH2):
    files_list.append(pandas.read_csv(fname2, dtype=datatypes))
for fname3 in glob.glob(DATA_PATH3):
    files_list.append(pandas.read_csv(fname3, dtype=datatypes))
for fname4 in glob.glob(DATA_PATH4):
    files_list.append(pandas.read_csv(fname4, dtype=datatypes))
    

In [52]:
def typeSafe(v):
    if v == "-999999" or v == "inf":
        return numpy.nan
    else:
        return float(v)

In [53]:
ddf = files_list.pop(0)
ddf['safe_ratio'] = ddf['dist_ratio'].apply(typeSafe)
ddf['dist_calc'] = ddf['dist_calc'].astype(float)
ddf['od_line_dist'] = ddf['od_line_dist'].astype(float)
ddf['matching_error'] = ddf['matching_error'].astype(float)
ddf['lon_o'] = ddf['lon_o'].astype(float)
ddf['lat_o'] = ddf['lat_o'].astype(float)
ddf['lon_d'] = ddf['lon_d'].astype(float)
ddf['lat_d'] = ddf['lat_d'].astype(float)
ddf['f_distance'] = ddf['f_distance'].astype(float)
ddf['dist_ratio'] = ddf['dist_ratio'].astype(float)
for f in files_list:
    f['safe_ratio'] = f['dist_ratio'].apply(typeSafe)
    f['dist_calc'] = f['dist_calc'].astype(float)
    f['od_line_dist'] = f['od_line_dist'].astype(float)
    f['matching_error'] = f['matching_error'].astype(float)
    f['lon_o'] = f['lon_o'].astype(float)
    f['lat_o'] = f['lat_o'].astype(float)
    f['lon_d'] = f['lon_d'].astype(float)
    f['lat_d'] = f['lat_d'].astype(float)
    f['f_distance'] = f['f_distance'].astype(float)
    f['dist_ratio'] = f['dist_ratio'].astype(float)
    ddf = ddf.append(f)

In [54]:
ddf.to_csv('C:\\Users\\bitas\\folders\\Research\\lime\peter\data\\detour\\all_aa.csv')

In [7]:
# DATA_PATH = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_00_140\\csv_files\\*.csv'
# DATA_PATH = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_141_320\\csv_files\\*.csv'
# DATA_PATH = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_321_384\\csv_files\\*.csv'
DATA_PATH = 'C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_384_500\\csv_files\\*.csv'

In [42]:

df_tot = pandas.DataFrame()

for fname in glob.glob(DATA_PATH):
    df = pandas.read_csv(fname)
    df_tot = df_tot.append(df)
    
print(len(df_tot))


8478


In [43]:
len(df_tot)

8478

In [44]:
numpy.percentile(df_tot['dist_ratio'], 0)

0.9837565774422328

In [45]:
df_loop = df_tot.loc[(df_tot['od_line_dist']<401) & (df_tot['dist_calc']>3*df_tot['od_line_dist'])]

In [46]:
len(df_loop)

1159

In [47]:
df_nonloop = df_tot[~df_tot['trip_id'].isin(df_loop['trip_id'])]

In [48]:
len(df_nonloop)

7319

In [49]:
len(df_nonloop[df_nonloop['dist_calc']>100])

7222

In [50]:
df_nonloop = df_nonloop[df_nonloop['dist_calc']>100]

In [51]:
df_nonloop[df_nonloop['dist_ratio']<0.99]

,trip_id,dist_calc,od_line_dist,matching_error,propulsion_type,matched_start,matched_end,lon_o,lat_o,lon_d,lat_d,f_coordinates,f_distance,f_duration,dist_ratio


In [27]:
df_nonloop = df_nonloop[df_nonloop['dist_ratio']>=0.99]

In [52]:
df_nonloop.to_csv('C:\\Users\\bitas\\folders\\Research\\lime\\peter\\data\\detour\\cr_384_500\\cr_384_500_nonloop.csv')